## Creación de la clase ternaria en SQL

### Dependencies & Setup

In [1]:
# %%bash
# !pip install duckdb
# !pip install jupysql
# !pip install duckdb-engine

In [ ]:
import pandas as pd
import duckdb
import os

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Create a connection object
connection = duckdb.connect()

# Pass the connection to the SQL magic command
%sql duckdb:///:memory:

### Uso de SQL con funciones analíticas

In [ ]:
%%sql
create or replace table competencia_03_crudo as
select
    *
from read_csv_auto('/home/santtedo/buckets/b1/datasets/competencia_03_crudo.csv.gz')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [7]:
## cambio el condicional según meme brandoni

In [ ]:
%%sql 
create or replace table competencia_03 as
with periodos as (
    select distinct foto_mes from competencia_03_crudo
), clientes as (
    select distinct numero_de_cliente from competencia_03_crudo
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        c.*
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , if(mes_2 = 1, 'CONTINUA', 
            if( mes_1 = 0, 'BAJA+1',
                if(mes_2=0, 'BAJA+2', null))) as clase_ternaria
    from todo t
    left join competencia_03_crudo c using (numero_de_cliente, foto_mes)
) select
  * EXCLUDE (mes_0, mes_1, mes_2)
from clase_ternaria
where mes_0 = 1

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


Imprimo toda la tabla

In [ ]:
%sql select * from competencia_03

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,249239632,202101,1,0,0,52,263,12.90,-1020.41,951.02,...,4.0,0.0,-57325.80,0.0,5785.0,13134.60,7.0,0.0,2744.82,CONTINUA
1,249239632,202102,1,0,0,52,264,849.59,617.02,1862.72,...,-3.0,0.0,-28492.69,0.0,5813.0,16111.71,9.0,0.0,3448.62,CONTINUA
2,249239632,202103,1,0,0,52,265,1954.04,3460.90,898.96,...,1.0,0.0,-35025.01,0.0,5844.0,11427.81,5.0,0.0,3131.91,CONTINUA
3,249239632,202104,1,0,0,52,266,2610.89,7231.49,1619.99,...,2.0,0.0,-29334.20,0.0,5874.0,18525.47,8.0,0.0,3835.71,CONTINUA
4,249239632,202105,1,0,0,52,267,2963.71,10969.09,1121.34,...,5.0,0.0,-37566.22,0.0,5905.0,10756.25,4.0,0.0,3624.57,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,1595193701,202105,0,0,0,51,1,0.00,0.00,0.00,...,NaN,0.0,NaN,NaN,1.0,NaN,NaN,NaN,0.00,NaN
981942,1595193701,202106,1,0,0,51,2,1176.95,1176.95,980.82,...,0.0,0.0,NaN,NaN,31.0,NaN,NaN,NaN,0.00,NaN
981943,1598206246,202106,1,0,0,31,1,9.09,9.09,0.00,...,0.0,0.0,NaN,NaN,7.0,NaN,NaN,NaN,0.00,NaN
981944,1598444941,202106,0,0,0,27,1,0.00,0.00,0.00,...,NaN,0.0,NaN,NaN,15.0,NaN,NaN,NaN,0.00,NaN


Revisión de caso

In [ ]:
%%sql
select
    numero_de_cliente
    , foto_mes
from competencia_03
where numero_de_cliente = 304051318
order by foto_mes

,numero_de_cliente,foto_mes
0,304051318,202101
1,304051318,202102
2,304051318,202104
3,304051318,202105
4,304051318,202106


Este caso se dio de baja en 202103 y luego retomó

In [ ]:
%%sql 
select numero_de_cliente, foto_mes, clase_ternaria 
from competencia_03
where numero_de_cliente = 304051318

,numero_de_cliente,foto_mes,clase_ternaria
0,304051318,202101,BAJA+2
1,304051318,202102,CONTINUA
2,304051318,202104,CONTINUA
3,304051318,202105,NaN
4,304051318,202106,NaN


Guardo la tabla

In [ ]:
%%sql 
COPY competencia_03 TO '/home/santtedo/buckets/b1/datasets/competencia_03_brandoni.csv.gz' (FORMAT CSV, HEADER)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success
